##Bubble Sort

In [28]:
%%writefile "bubble_Parallel.cpp"

#include <iostream>
#include <chrono>
#include <omp.h>
#include <stdio.h>
#include <random>

using namespace std;
using namespace std::chrono;

int random_in_range( int minimum, int maximum )
{
  thread_local std::ranlux48 rng( 
    std::chrono::system_clock::now().time_since_epoch().count() );
  return std::uniform_int_distribution <int> ( minimum, maximum )( rng );
}


void bubbleSortSerial(int a[], int n)
{
	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();

	for (int i = 0; i < n-1; i++)
	{
		for (int j = 0; j < n-1; j++)
		{
			if(a[j] > a[j+1])
			{
				int temp = a[j];
				a[j] = a[j+1];
				a[j+1] = temp;
			}
		}
	}
	endtime = system_clock::now();
	duration <double> time= endtime - starttime;
	cout<<"Time for serial: "<<1000*time.count()<<" milliseconds"<<endl;
}

void bubbleSortOddEven(int b[], int n)
{
	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	int pass;

	for(int i = 0 ; i < n-1 ; i++)
	{
		pass = i % 2;
		for (int j = pass ; j < n-1 ; j+=2)
		{
			if(b[j]>b[j+1])
			{
				int temp = b[j];
				b[j] = b[j+1];
				b[j+1]=temp;	
			}
		}
	}
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for Bubble sort (Odd Even Transposition): "<<1000*time.count()<<" milliseconds"<<endl;
}

void bubbleSortParallel(int b[], int n)
{
	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	int pass;

	omp_set_num_threads(2);

	for(int i = 0 ; i < n-1 ; i++)
	{
		pass = i % 2;
		#pragma omp parallel for
		for (int j = pass ; j < n-1 ; j+=2)
		{
			if(b[j]>b[j+1])
			{
				int temp = b[j];
				b[j] = b[j+1];
				b[j+1]=temp;	
			}
		}
	}
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for Parallel: "<<1000*time.count()<<" milliseconds"<<endl;
}

void init_array(int *arr1, int *arr2, int *arr3, int n) {
	for(int i = 0 ; i < n ; i++) {
		arr1[i] = arr2[i] = arr3[i] = random_in_range(10,99);
	}
}

void print_array(int *arr, int n) {
	for(int i = 0 ; i < n ; i++) {
		cout<<arr[i]<<" ";
	}
}

int main()
{
	int n = 50;
	int *a, *b, *c;
	a = new int[n];
	b = new int[n];
	c = new int[n];
	init_array(a, b, c, n);
	cout<<"Initial vector: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;

	bubbleSortSerial(a,n);
	cout<<"Result after serial bubble sort: "<<endl;
	print_array(a, n);

	cout<<endl;
	cout<<endl;

	bubbleSortOddEven(c,n);
	cout<<"Result after odd-even sort: "<<endl;
	print_array(c, n);

	cout<<endl;
	cout<<endl;
	bubbleSortParallel(b,n);
	cout<<"Result after parallel bubble sort: "<<endl;
	print_array(b, n);
	cout<<endl;

	return 0;
}


Overwriting bubble_Parallel.cpp


In [29]:
!g++ -fopenmp bubble_Parallel.cpp -o bubble_Parallel

In [30]:
!./bubble_Parallel

Initial vector: 
17 45 75 79 25 53 20 27 59 85 78 42 10 76 94 79 56 22 89 28 36 63 34 60 16 95 57 23 69 86 21 90 89 62 84 24 59 84 98 87 44 87 33 31 21 55 18 14 54 25 

Time for serial: 0.019842 milliseconds
Result after serial bubble sort: 
10 14 16 17 18 20 21 21 22 23 24 25 25 27 28 31 33 34 36 42 44 45 53 54 55 56 57 59 59 60 62 63 69 75 76 78 79 79 84 84 85 86 87 87 89 89 90 94 95 98 

Time for Bubble sort (Odd Even Transposition): 0.011963 milliseconds
Result after odd-even sort: 
10 14 16 17 18 20 21 21 22 23 24 25 25 27 28 31 33 34 36 42 44 45 53 54 55 56 57 59 59 60 62 63 69 75 76 78 79 79 84 84 85 86 87 87 89 89 90 94 95 98 

Time for Parallel: 7.96642 milliseconds
Result after parallel bubble sort: 
10 14 16 17 18 20 21 21 22 23 24 25 25 27 28 31 33 34 36 42 44 45 53 54 55 56 57 59 59 60 62 63 69 75 76 78 79 79 84 84 85 86 87 87 89 89 90 94 95 98 


##Merge Sort

In [31]:
%%writefile "merge_Parallel.cpp"

#include <iostream>
#include <chrono>
#include <omp.h>
#include <stdio.h>
#include <random>
#include <string.h>
#include <stdlib.h>

using namespace std;
using namespace std::chrono;

int random_in_range( int minimum, int maximum )
{
  thread_local std::ranlux48 rng( 
    std::chrono::system_clock::now().time_since_epoch().count() );
  return std::uniform_int_distribution <int> ( minimum, maximum )( rng );
}

void merge(int *c1, int *c2, int *p, int cl1, int cl2) {
	int i = 0, j = 0, k = 0;
	while(i < cl1 && j < cl2) {
		if(c1[i] < c2[j]) {
			p[k] = c1[i];
			i++;
			k++;
		}
		else {
			p[k] = c2[j];
			j++;
			k++;
		}
	}
	while(i < cl1) {
		p[k] = c1[i];
		i++;
		k++;
	}
	while(j < cl2) {
		p[k] = c2[j];
		j++;
		k++;
	}
}

void mergeSortSerial(int *arr, int length) {
	if(length <= 1)
		return;

	int mid = length / 2;
	
	int *left = new int[mid];
	int *right = new int[length-mid];

	for(int i = 0 ; i < mid ; i++) {
		left[i] = arr[i];
	}
	for(int i = mid ; i < length ; i++) {
		right[i-mid] = arr[i];
	}
	mergeSortSerial(left, mid);
	mergeSortSerial(right, length-mid);
	merge(left, right, arr, mid, length-mid);
}

void mergeSortParallel(int *arr, int length) {
	if(length <= 1)
		return;
	int mid = length / 2;
	int *left, *right;
	left = new int[mid];
	right = new int[length-mid];

	#pragma omp task firstprivate(left)
	{
		
		for(int i = 0 ; i < mid ; i++) {
			left[i] = arr[i];
		}
		mergeSortParallel(left, mid);
	}
	
	
	#pragma omp task firstprivate(right)
	{
		
		for(int i = mid ; i < length ; i++) {
			right[i-mid] = arr[i];
		}
	
		mergeSortParallel(right, length-mid);		
	}


	// #pragma omp parallel sections
	// {
	// 	#pragma omp section
	// 	{
	// 		left = new int[mid];
	// 		for(int i = 0 ; i < mid ; i++) {
	// 			left[i] = arr[i];
	// 		}
	// 		mergeSortParallel(left, mid);
	// 	}
	// 	#pragma omp section
	// 	{
	// 		right = new int[length-mid];
	// 		for(int i = mid ; i < length ; i++) {
	// 			right[i-mid] = arr[i];
	// 		}
	// 		mergeSortParallel(right, length-mid);
	// 	}
	// }
	#pragma omp taskwait
	merge(left, right, arr, mid, length-mid);
}

void init_array(int *arr1, int *arr2, int n) {
	for(int i = 0 ; i < n ; i++) {
		arr1[i] = arr2[i] = random_in_range(10,99);
	}
}

void print_array(int *arr, int n) {
	for(int i = 0 ; i < n ; i++) {
		cout<<arr[i]<<" ";
	}
}

int main() {
	int n = 50;
	int *a, *b;
	a = new int[n];
	b = new int[n];

	init_array(a, b, n);
	cout<<"Initial vector: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;

	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	mergeSortSerial(a, n);
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for serial: "<<1000*time.count()<<" milliseconds"<<endl;
	cout<<"Result after serial merge sort: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;


	starttime = system_clock::now();
	mergeSortParallel(b, n);
	endtime = system_clock::now();
	time = endtime - starttime;
	cout<<"Time for parallel: "<<1000*time.count()<<" milliseconds"<<endl;
	cout<<"Result after parallel merge sort: "<<endl;
	print_array(b, n);

	return 0;	
}

Writing merge_Parallel.cpp


In [32]:
!g++ -fopenmp merge_Parallel.cpp -o merge_Parallel

In [33]:
!./merge_Parallel

Initial vector: 
68 54 83 43 92 17 65 48 72 40 39 30 16 51 60 39 82 72 88 63 50 24 50 30 70 76 49 99 20 94 75 68 52 83 59 27 94 49 54 80 99 15 93 58 89 73 27 18 56 43 

Time for serial: 0.013729 milliseconds
Result after serial merge sort: 
15 16 17 18 20 24 27 27 30 30 39 39 40 43 43 48 49 49 50 50 51 52 54 54 56 58 59 60 63 65 68 68 70 72 72 73 75 76 80 82 83 83 88 89 92 93 94 94 99 99 

Time for parallel: 0.035115 milliseconds
Result after parallel merge sort: 
15 16 17 18 20 24 27 27 30 30 39 39 40 43 43 48 49 49 50 50 51 52 54 54 56 58 59 60 63 65 68 68 70 72 72 73 75 76 80 82 83 83 88 89 92 93 94 94 99 99 